In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from ultralytics import YOLO


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.2 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Dataset paths
BASE_DATASET = "/content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset"
IMG_DIR = os.path.join(BASE_DATASET, "all_images")
XML_DIR = os.path.join(BASE_DATASET, "all_xml")

# YOLO model
YOLO_WEIGHTS = "/content/drive/MyDrive/ANPR_Model_Full/weights/best.pt"

print("Images:", len(os.listdir(IMG_DIR)))
print("XMLs  :", len(os.listdir(XML_DIR)))


Images: 1339
XMLs  : 1338


In [4]:
def parse_xml(xml_path):
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        filename = root.find("filename").text
        size = root.find("size")
        w = int(size.find("width").text)
        h = int(size.find("height").text)

        objects = []
        for obj in root.findall("object"):
            bnd = obj.find("bndbox")
            bbox = [
                int(float(bnd.find("xmin").text)),
                int(float(bnd.find("ymin").text)),
                int(float(bnd.find("xmax").text)),
                int(float(bnd.find("ymax").text))
            ]
            objects.append({"bbox": bbox})

        return {
            "filename": filename,
            "image_size": (h, w),
            "objects": objects
        }

    except:
        return None


In [5]:
gt_data = {}
skipped = 0

for xml_file in os.listdir(XML_DIR):
    if not xml_file.endswith(".xml"):
        continue

    parsed = parse_xml(os.path.join(XML_DIR, xml_file))
    if parsed is None or len(parsed["objects"]) == 0:
        skipped += 1
        continue

    gt_data[parsed["filename"]] = parsed["objects"][0]["bbox"]

print("✅ GT samples:", len(gt_data))
print("⚠️ Skipped XMLs:", skipped)


✅ GT samples: 1338
⚠️ Skipped XMLs: 0


In [6]:
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    if interArea == 0:
        return 0.0

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    return interArea / float(boxAArea + boxBArea - interArea)


In [12]:
!mkdir -p /content/yolo_eval_images
!cp /content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset/all_images/* /content/yolo_eval_images/


In [14]:
from tqdm import tqdm

LOCAL_IMG_DIR = "/content/yolo_eval_images"

results_log = []
iou_values = []
detected = 0

model = YOLO(YOLO_WEIGHTS) # Initialize the YOLO model

image_list = list(gt_data.keys())

for image_name in tqdm(image_list):
    img_path = os.path.join(LOCAL_IMG_DIR, image_name)
    if not os.path.exists(img_path):
        continue

    image = cv2.imread(img_path)
    if image is None:
        continue

    preds = model.predict(
        image,
        conf=0.25,
        iou=0.5,
        device='cpu',
        verbose=False
    )[0]

    if preds.boxes is None or len(preds.boxes) == 0:
        results_log.append({
            "image": image_name,
            "detected": False,
            "iou": 0.0,
            "error": "DET_MISS"
        })
        continue

    boxes = preds.boxes.xyxy.cpu().numpy()
    scores = preds.boxes.conf.cpu().numpy()
    best_idx = scores.argmax()
    yolo_box = boxes[best_idx].astype(int).tolist()

    gt_box = gt_data[image_name]
    iou = compute_iou(yolo_box, gt_box)

    iou_values.append(iou)
    detected += 1

    results_log.append({
        "image": image_name,
        "detected": True,
        "iou": round(iou, 4),
        "error": None if iou >= 0.5 else "DET_POOR_IOU"
    })

100%|██████████| 1338/1338 [29:45<00:00,  1.33s/it]


In [15]:
total = len(gt_data)
recall = detected / total
mean_iou = sum(iou_values) / len(iou_values)

print("📊 YOLO Detection Metrics")
print("-------------------------")
print(f"Total images     : {total}")
print(f"Detected plates  : {detected}")
print(f"Recall           : {recall:.4f}")
print(f"Mean IoU         : {mean_iou:.4f}")


📊 YOLO Detection Metrics
-------------------------
Total images     : 1338
Detected plates  : 1335
Recall           : 0.9978
Mean IoU         : 0.9335
